In [1]:
import os
from pathlib import Path
import shutil

import teehr

teehr.__version__

'0.6.0dev4'

In [3]:
from teehr.evaluation.spark_session_utils import create_spark_session

In [4]:
spark = create_spark_session(
    start_spark_cluster=True,
    executor_instances=20,
    executor_memory="16g",
    executor_cores=4,
)

INFO:teehr.evaluation.spark_session_utils:🚀 Creating Spark session: TEEHR Evaluation
INFO:teehr.evaluation.spark_session_utils:📦 Configuring Spark cluster with container image: None
INFO:teehr.evaluation.spark_session_utils:🔍 Initial spark namespace from ENV: teehr-hub
INFO:teehr.evaluation.spark_session_utils:🔍 Connecting to Kubernetes API: https://172.20.0.1:443
INFO:teehr.evaluation.spark_session_utils:🎯 Executor namespace: teehr-hub
INFO:teehr.evaluation.spark_session_utils:🔐 Executor service account: spark (in teehr-hub)
INFO:teehr.evaluation.spark_session_utils:🔐 Using in-cluster authentication
INFO:teehr.evaluation.spark_session_utils:🔗 Setting driver host to pod IP: 10.0.2.254
INFO:teehr.evaluation.spark_session_utils:✅ Spark cluster configuration successful!
INFO:teehr.evaluation.spark_session_utils:   - Executor instances: 20
INFO:teehr.evaluation.spark_session_utils:   - Executor memory: 16g
INFO:teehr.evaluation.spark_session_utils:   - Executor cores: 4
INFO:teehr.evaluati

In [5]:
%%time
dir_path = "/data/temp_warehouse"

ev = teehr.Evaluation(
    spark=spark,
    dir_path=dir_path,
    check_evaluation_version=False,
)

INFO:teehr.evaluation.evaluation:Using provided Spark session.
INFO:teehr.evaluation.evaluation:Active catalog set to local.


CPU times: user 5.19 ms, sys: 71 μs, total: 5.27 ms
Wall time: 139 ms


In [6]:
ev.set_active_catalog("remote")

ev.active_catalog

INFO:teehr.evaluation.evaluation:Active catalog set to remote.


RemoteCatalog(warehouse_dir='s3://dev-teehr-iceberg-warehouse/', catalog_name='iceberg', namespace_name='teehr', catalog_type='rest', catalog_uri='http://iceberg-rest:8181')

In [8]:
from teehr import DeterministicMetrics as dm
from teehr import Signatures as s

In [16]:
sdf = ev.metrics(table_name="joined_simulation_timeseries").query(
    group_by=["primary_location_id", "configuration_name", "variable_name", "unit_name"],
    order_by=["primary_location_id", "configuration_name", "variable_name", "unit_name"],
    include_metrics=[
        s.Count(),
        s.Average(),
        dm.RelativeBias(),
        dm.NashSutcliffeEfficiency(),
        dm.KlingGuptaEfficiency()
    ]
).to_sdf()

INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.joined_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.joined_timeseries.
INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.joined_simulation_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.joined_simulation_timeseries.
INFO:teehr.evaluation.metrics:Calculating performance metrics.


In [17]:
sdf.createTempView("metrics")

In [18]:
sdf = ev.spark.sql("""
SELECT m.*, l.* FROM metrics m JOIN iceberg.teehr.locations l ON l.id = m.primary_location_id
""")
sdf = sdf.drop("id")

In [19]:
sdf.show()

+-------------------+-------------------+--------------------+---------+--------+----------+-------------+-------------------------+----------------------+--------------------+--------------------+
|primary_location_id| configuration_name|       variable_name|unit_name|   count|   average|relative_bias|nash_sutcliffe_efficiency|kling_gupta_efficiency|                name|            geometry|
+-------------------+-------------------+--------------------+---------+--------+----------+-------------+-------------------------+----------------------+--------------------+--------------------+
|      usgs-01013500|nwm30_retrospective|streamflow_daily_...|    m^3/s|  4821.0| 55.141956| -0.055877265|               0.55584747|            0.42716876|Fish River near F...|[01 01 00 00 00 D...|
|      usgs-01038000|nwm30_retrospective|streamflow_hourly...|    m^3/s|236169.0|  8.327773| -0.058371633|               0.68670714|             0.7095444|Sheepscot River a...|[01 01 00 00 00 7...|
|      usg

In [20]:
ev.write.to_warehouse(source_data=sdf, table_name="sim_metrics_by_location", write_mode="create_or_replace")

In [21]:
spark.stop()